<a href="https://colab.research.google.com/github/RiccardoRubini93/ML-AI-cookbook/blob/main/similarity_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openpyxl
!pip install spacy
!pip install nltk

In [2]:
import pandas as pd
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import files

In [25]:
business_glossary_df_ori = pd.read_csv('BUSINESS_GLOSSARY.csv',sep=';')
interfacce_df = pd.read_csv('Interfacce_normalized.csv',sep=';')

In [ ]:
business_glossary_df = business_glossary_df_ori

In [15]:
business_glossary_df['BUSINESS TERM'] = business_glossary_df['BUSINESS TERM'] + ' ' + business_glossary_df['DEFINIZIONE']

In [12]:
business_glossary_df.head(10)

,ID,BUSINESS TERM,DEFINIZIONE,FONTE,Unnamed: 4
0,1,agenzia mandato uffici periferici dell’impresa...,uffici periferici dell’impresa assicurativa al...,IVASS,NaN
1,2,agenzia economia gerenza uffici delle compagni...,uffici delle compagnie assicurative che promuo...,IVASS,NaN
2,3,aliquota partecipazione utili quota determinat...,quota determinata contrattualmente di attribuz...,IVASS,NaN
3,4,aliquota retrocessione contraente percentuale ...,percentuale del rendimento realizzato dalla ge...,IVASS,NaN
4,5,"alta direzione l’amministratore delegato, il d...","l’amministratore delegato, il direttore genera...",IVASS,NaN
5,6,antidurata tempo decorso dalla stipula del con...,tempo decorso dalla stipula del contratto,IVASS,NaN
6,7,archivio integrato antifrode aia dicembre 2012...,base dati istituita presso l’IVASS dall’art. 2...,IVASS,NaN
7,8,assicurazione complementare estensione di gar...,estensione di garanzia finalizzata a potenzia...,IVASS,NaN
8,9,assicurazione danni le assicurazioni di cui al...,"le assicurazioni di cui all'art. 2, comma 3, d...",IVASS,NaN
9,10,assicurazione obbligatoria responsabilità civi...,l’assicurazione obbligatoria di ogni responsab...,IVASS,NaN


In [13]:
print(interfacce_df.shape)
print(business_glossary_df.shape)

(4455, 6)
(2779, 5)


In [16]:
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('italian'))

def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]
    return ' '.join(tokens)

business_glossary_df['BUSINESS TERM'] = business_glossary_df['BUSINESS TERM'].apply(tokenize_and_remove_stopwords)
interfacce_df['CAMPO FISICO'] = interfacce_df['CAMPO FISICO'].apply(tokenize_and_remove_stopwords)
#interfacce_df['DESCRIZIONE'] = interfacce_df['DESCRIZIONE'].apply(tokenize_and_remove_stopwords)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_business = tfidf_vectorizer.fit_transform(business_glossary_df['BUSINESS TERM'])
tfidf_matrix_interfacce = tfidf_vectorizer.transform(interfacce_df['CAMPO FISICO'])

In [18]:
#cosine_similarities = cosine_similarity(tfidf_matrix_interfacce, tfidf_matrix_business)
cosine_similarities = cosine_similarity(tfidf_matrix_business,tfidf_matrix_interfacce)

In [19]:
cosine_similarities.shape

(2779, 4455)

In [ ]:
results = []
for idx, row in enumerate(interfacce_df.iterrows()):
  similar_business_term_idx = cosine_similarities[idx].argmax()
  business_term = business_glossary_df.iloc[similar_business_term_idx]['BUSINESS TERM']
  campo_fisico = row[1]['CAMPO FISICO']
  results.append((campo_fisico, business_term))

In [26]:
results = []
for idx, row in enumerate(business_glossary_df_ori.iterrows()):
  similar_business_term_idx = cosine_similarities[idx].argmax()
  business_term = row[1]['BUSINESS TERM']
  campo_fisico = interfacce_df.iloc[similar_business_term_idx]['CAMPO FISICO']
  results.append((campo_fisico, business_term))

In [28]:
results_df = pd.DataFrame(results, columns=['CAMPO FISICO', 'BUSINESS TERM'])
results_df.head(100)

,CAMPO FISICO,BUSINESS TERM
0,Mandato,AGENZIA CON MANDATO
1,Agenzia,AGENZIA IN ECONOMIA E GERENZA
2,Tasso tecnico,ALIQUOTA DI PARTECIPAZIONE AGLI UTILI
3,COSTI GESTIONE,ALIQUOTA DI RETROCESSIONE CONTRAENTE
4,Tipo record,ALTA DIREZIONE
...,...,...
95,Tipo record,GRUPPI ASSICURATIVI ATTIVI A LIVELLO INTERNAZI...
96,Attività,GRUPPI RILEVANTI AI FINI DI STABILITÀ FINANZIA...
97,TIPO,GRUPPO ASSICURATIVO
98,NUMERO EMESSO DI RIMBORSO-RECESSO-REVOCA-ESTIN...,IMPORTI A PREZZI COSTANTI


In [28]:
#define score: number of recognised terms / total
results_df.shape[0]

4455

In [33]:
df = results_df

In [41]:
df_filtered = df[(df['CAMPO FISICO'] != '') & (df['BUSINESS TERM'] != '')]

In [42]:
df_filtered.to_csv('result.csv')